In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_bokeh
import geopandas as gpd
import datetime

# pd.set_option('plotting.backend', 'pandas_bokeh')
pandas_bokeh.output_notebook()


Loading BokehJS ...

# Svetová mapa

In [18]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv", parse_dates=["date"]
).loc[
    lambda df: ~df["iso_code"].str.contains("_"),
    ["iso_code", "location","date", "total_cases","total_cases_per_million", "total_deaths", "total_deaths_per_million", "new_cases"],
]

grouped_data = (df.sort_values('date')
                    .groupby('iso_code')
                    .tail(1)
                    .drop(columns=["new_cases"]))


In [19]:
grouped_data = grouped_data.set_index('iso_code').join(
    (df[df.date > datetime.datetime.now() - pd.to_timedelta("30day")] \
    .groupby('iso_code')
    .sum('new_cases')
    .rename(columns={'new_cases': 'cases_last_30d'}))['cases_last_30d'])


In [20]:
gdf = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")).loc[:, ["iso_a3", "name", "geometry"]]
gdf.loc[gdf.name == 'France', 'iso_a3'] = 'FRA'    
gdf.loc[gdf.name == 'Norway', 'iso_a3'] = 'NOR'    

gdf = gdf.merge(grouped_data, left_on="iso_a3", right_on="iso_code")
gdf['date'] = gdf['date'].dt.strftime('%Y-%m-%d')


In [21]:
gdf.plot_bokeh(
        dropdown=["total_deaths", "total_cases", "cases_last_30d","total_cases_per_million","total_deaths_per_million"],
        show_colorbar=False,
        colormap="RdYlGn",
        hovertool_string="<h3>@name</h3><pre>@Colormap{0.0a}</pre>",
        figsize=(1200, 600),
    )

/home/egruy/.local/lib/python3.8/site-packages/pandas_bokeh/geoplot.py:118: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:


Column(id='4733', ...)

# Porovnanie krajín

In [57]:
data = (pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv", parse_dates=['date'])
        .loc[
    lambda df: ~df["iso_code"].str.contains("_")
].set_index('date'))


In [23]:
plt.figure()
plt.rcParams.update({'font.size': 22}) # must set in top
plt.rcParams.update({'figure.figsize': (24,10)}) # must set in top

<Figure size 1728x720 with 0 Axes>

In [58]:
data[data['iso_code'] == "DEU"].head(3)

,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
date,,,,,,,,,,,,,,,,,,,,,
2020-01-27,DEU,Europe,Germany,1.0,1.0,NaN,NaN,NaN,NaN,0.012,...,28.2,33.1,NaN,8.0,81.33,0.947,NaN,NaN,NaN,NaN
2020-01-28,DEU,Europe,Germany,4.0,3.0,NaN,NaN,NaN,NaN,0.048,...,28.2,33.1,NaN,8.0,81.33,0.947,NaN,NaN,NaN,NaN
2020-01-29,DEU,Europe,Germany,4.0,0.0,NaN,NaN,NaN,NaN,0.048,...,28.2,33.1,NaN,8.0,81.33,0.947,NaN,NaN,NaN,NaN


### TOP 5 VS SLOVENSKO

In [146]:
def get_top7(data, column):
    ranked = (data.groupby('location')
        .max('date')
        .sort_values(column, ascending=False)
        .reset_index())

    ranked.index = ranked.index + 1
    ranked.index.names = ["Rank"]

    return ranked.head(7).append(ranked[ranked['location'].isin(["Slovakia","Czechia","Germany","Sweden"])])[['location', column]]


In [147]:
get_top7(data, 'total_deaths')

,location,total_deaths
Rank,,
1,United States,985482.0
2,Brazil,661475.0
3,India,521685.0
4,Russia,364011.0
5,Mexico,323691.0
6,Peru,212471.0
7,United Kingdom,169916.0
13,Germany,131715.0
25,Czechia,39880.0


In [148]:
get_top7(data, 'total_cases')

,location,total_cases
Rank,,
1,United States,80399486.0
2,India,43035271.0
3,Brazil,30146769.0
4,France,26893286.0
5,Germany,22684849.0
6,United Kingdom,21619833.0
7,Russia,17720977.0
5,Germany,22684849.0
28,Czechia,3862322.0


In [149]:
get_top7(data, 'total_deaths_per_million')

,location,total_deaths_per_million
Rank,,
1,Peru,6369.146
2,Bulgaria,5320.260
3,Bosnia and Herzegovina,4820.652
4,Hungary,4745.716
5,North Macedonia,4438.552
6,Montenegro,4308.567
7,Georgia,4213.808
9,Czechia,3718.570
10,Slovakia,3587.453


In [150]:
get_top7(data, 'total_cases_per_million')

,location,total_cases_per_million
Rank,,
1,Faeroe Islands,706541.904
2,Denmark,530409.740
3,Andorra,521343.434
4,Gibraltar,511590.633
5,Cyprus,509066.356
6,Iceland,497198.963
7,Slovenia,474675.558
10,Slovakia,456619.511
24,Czechia,360138.273
